In [1]:
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, PrimaryKeyConstraint, func,inspect,text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
import sqlite3
from sqlalchemy.ext.automap import automap_base

In [2]:
url="https://api.census.gov/data/2019/pep/charagegroups?"


In [3]:
all_population="000"
age_65_up="026"
query_url=url+ "get=NAME,POP&for=county:*&in=state:12&AGEGROUP="
all_pop_query=query_url+all_population
age_65_up_query=query_url+age_65_up
results_cols=["County, State","Population","Age Group","State Code","County Code"]

In [4]:
all_pop_results=requests.get(all_pop_query).json()
age_65_up_results=requests.get(age_65_up_query).json()

In [5]:
age_65_up_df=pd.DataFrame(age_65_up_results,columns=results_cols)
age_65_up_df.columns=age_65_up_df.iloc[0]
age_65_up_df=age_65_up_df.tail(-1)
age_65_up_df

,NAME,POP,AGEGROUP,state,county
1,"Washington County, Florida",4380,26,12,133
2,"Duval County, Florida",138594,26,12,031
3,"Bradford County, Florida",5064,26,12,007
4,"Brevard County, Florida",145752,26,12,009
5,"Clay County, Florida",35844,26,12,019
...,...,...,...,...,...
63,"Hernando County, Florida",53348,26,12,053
64,"Hillsborough County, Florida",214035,26,12,057
65,"Okaloosa County, Florida",34376,26,12,091
66,"Taylor County, Florida",4525,26,12,123


In [6]:
all_pop_df=pd.DataFrame(all_pop_results,columns=results_cols)
all_pop_df.columns=all_pop_df.iloc[0]
all_pop_df=all_pop_df.tail(-1)

In [7]:
pop_percent_df=pd.merge(all_pop_df,age_65_up_df,on='county',how='outer')

In [8]:
pop_percent_df[['County','State']]=pop_percent_df['NAME_x'].str.split(', ',expand=True)
pop_percent_df.drop(columns=['NAME_x','NAME_y','AGEGROUP_x','AGEGROUP_y','state_x','state_y','State'],inplace=True)

In [9]:
pop_percent_df=pop_percent_df.rename(columns={'county':'County_ID','POP_x':'Total_Population','POP_y':'Elderly_Population','state':'State_ID'})
data_types={'Total_Population':'int','County_ID':'int','Elderly_Population':'int','County':'str'}
pop_percent_df=pop_percent_df.astype(data_types)


In [10]:
pop_percent_df['Percent_Elderly']=round((pop_percent_df['Elderly_Population']/pop_percent_df['Total_Population'])*100,2)
# pop_percent_df['Percent_Elderly'] = pop_percent_df['Percent_Elderly'].map('{:.2f}'.format)
pop_percent_df['County'] = pop_percent_df['County'].str.replace(' County', '')

In [11]:
county_table=pop_percent_df[['County_ID','County']]

In [12]:
city_county_df=pd.read_csv('Florida City County and Coordinates.csv')
city_county_df=city_county_df.rename(columns={"CITY":"City","COUNTY":"County","LATITUDE":"Latitude","LONGITUDE":"Longitude"})
pop_percent_df.dtypes


0
Total_Population        int32
County_ID               int32
Elderly_Population      int32
County                 object
Percent_Elderly       float64
dtype: object

In [19]:
# # Create the SQLAlchemy engine
# engine = create_engine("sqlite:///../Resources/demographics.sqlite")

# # Create a base class
# Base = automap_base()

# # Reflect the database schema into the base class
# Base.prepare(autoload_with=engine)
conn = sqlite3.connect('florida_full_db.sqlite')
pop_percent_df.to_sql('elderly_people', conn, index=False, if_exists='replace', dtype={'County_ID': 'INTEGER PRIMARY KEY'})

67

In [14]:
# Import the dependencies.
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect


#################################################
# Database Setup
#################################################

engine = create_engine("sqlite:///your_database.sqlite")

# reflect an existing database into a new model
Base=automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# Save references to each table
data=Base.classes.keys()
conn = engine.connect()
print(engine)


Engine(sqlite:///your_database.sqlite)


In [15]:
data=pd.read_sql('SELECT * FROM your_table_name LIMIT 5', conn)


In [16]:
data

,Total_Population,County_ID,Elderly_Population,County,Percent_Elderly
0,25473,133,4380,Washington,17.19
1,957755,31,138594,Duval,14.47
2,28201,7,5064,Bradford,17.96
3,601942,9,145752,Brevard,24.21
4,219252,19,35844,Clay,16.35


In [17]:
# Base = declarative_base()
# class elderly(Base):
#     __tablename__ = 'elderly'

#     id = Column(Integer, primary_key=True, autoincrement=True)
#     county= Column(String)
#     elderly_population= Column(Integer)
#     total_population=Column(Integer)
#     percent_elderly=Column(Float)

# engine = create_engine('sqlite:///fl_full_db.sqlite')
# Base.metadata.create_all(engine)

# Session = sessionmaker(bind=engine)
# session = Session()

# for _, row in pop_percent_df.iterrows():
#     next_location = elderly(
#         county=row['County'],
#         elderly_population=row['Elderly_Population'],        
#         total_population=row['Total_Population'],
#         percent_elderly=row['Percent_Elderly'])
#     session.add(next_location)

# session.commit()
# session.close()